In [0]:
from delta.tables import DeltaTable
from typing import Dict,Tuple
Table_name = "sesion_11.gold.ventas"
def get_metrics(
    Table_name: str
)-> Tuple[Dict[str,int],str]:
    
    delta_table = DeltaTable.forName(spark, Table_name)
    history_df = delta_table.history()
    
    for row in history_df.collect():
        #print(row.operation)
        if row.operation in ("MERGE","WRITE"):
            row_metrics = row.asDict().get("operationMetrics",{})

            numeric_metrics = {}
            for k,v in raw_metrics.items():
                #print(f"{k} : {v}")
                numeric_metrics[k] = int(v)

            return numeric_metrics,row.operation

get_metrics("sesion_11.gold.ventas")


In [0]:
from pyspark.sql.functions import lit,current_timestamp,col

def insert_metrics(
    metrics_tuple: Tuple[Dict[str,int],str]
)-> None:
    metrics,Table_name = metrics_tuple
    job_id=0
    job_run_id=0
    task_run_id=0
    job_start_time = '2023-01-01 00:00:00'

    catalog, schema, table = Table_name.split(".")
    df_metrics =(
        spark.createDataFrame([metrics])
        .withColumn("job_id",lit(job_id))
        .withColumn("job_run_id",lit(job_run_id))
        .withColumn("task_run_id",lit(task_run_id))
        .withColumn("job_start_time",lit(job_start_time)).cast("timestamp")
        .withColumn("job_end_time",current_timestamp())
        .withColumn("job_duration_seconds",col("job_end_time").cast("long")-col("job_start_time").cast("long") )
        .withColumn("file_bytes",col("numTargetBytesAdded"))
        .withColumn("job_status",
                    when(col("file_bytes")>0,lit("Success") )
                    .otherwise(lit("Failed"))
                    )
        .withColumn("table",lit(table))
        .withColumn("layer",lit(chema))
        .withColumn("row_in",col("numTargetRowsInserted"))
        .withColumn("row_inserted",col("numTargetRowsInserted"))
        .withColumn("row_update",col("numTargetRowsUpdated"))
        .withColumn("row_delete",col("numTargetRowsDeletedd"))
        .withColumn("merge_duration_seconds",lit("executionTimeMs")/lit(1000))
        .select(
            "job_id",
            "job_run_id",
            "task_run_id",
            "job_start_time",
            "job_end_time",
            "job_duration_seconds",
            "file_bytes",
            "job_status",
            "table",
            "layer",
            "row_in",
            "row_inserted",
            "row_update",
            "row_delete",
            "merge_duration_seconds"

        )
        
    )
    
    #display(df_metrics)
    delta_table = DeltaTable.forName(spark, "sesion_11.gold.ventas")

    merge = (
        delta_table.alias("m")
        .merge(
            df_metrics.alias("in"),
            lit(False) 
        )
        .whenNotMatchedInsert